<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/main/SNOWPACKforPatrollers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNOWPACK Model Setup and Execution

This notebook guides you through the process of setting up and running the SNOWPACK model. It includes steps for installing necessary libraries, compiling the SNOWPACK and MeteoIO code, configuring the model, fetching meteorological data from historical weather models, and running SNOWPACK.

# The following cells below setup SNOWPACK AND MeteoIO and the PATH strcuture to run it. They should not be edited. Pressing the play button below will run them all.

In [ ]:
# @title Install environment updates
!apt-get update
!apt-get install -y build-essential cmake git liblapack-dev numdiff

%pip install openmeteo-requests requests-cache retry-requests pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from pathlib import Path
import openmeteo_requests
import requests_cache
from retry_requests import retry
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,799 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Fetched 5,780 kB in 3s

In [ ]:
# @title Download SNOWPACK
!git clone https://github.com/snowpack-model/snowpack.git


Cloning into 'snowpack'...
remote: Enumerating objects: 15270, done.
remote: Counting objects: 100% (989/989), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 15270 (delta 745), reused 733 (delta 660), pack-reused 14281 (from 2)
Receiving objects: 100% (15270/15270), 84.87 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (10662/10662), done.
Updating files: 100% (1756/1756), done.


In [ ]:
# @title Make  ~/usr folder
!mkdir ~/usr/

In [ ]:
# @title Compile MeteoIO  (Runs about 6 Mins)
!cd /content/snowpack/Source/meteoio/ && \
cmake -S . -B build-meteoio -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=$HOME/usr && \
cmake --build build-meteoio -j2 && \
cmake --install build-meteoio

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS - Failed
-- Performing Test HAVE_FEENABLE
-- Performing Test HAVE_FEENABLE - Success
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (3.2s)
-- Generating done (0.2s)
-- Build files have been written to: /content/snowpack/Source/meteoio/build-meteoio
[  1%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ct

In [ ]:
# @title Compile SNOWPACK (Runs about 2 mins)

!cd /content/snowpack/Source/snowpack/ && \
cmake -S . -B build-snowpack -DCMAKE_BUILD_TYPE=Release \
      -DCMAKE_PREFIX_PATH=$HOME/usr \
      -DDEBUG_ARITHM=OFF \
      -DENABLE_LAPACK=ON \
      -DMETEOIO_LIBRARY=$HOME/usr/lib/libmeteoio.so && \
cmake --build build-snowpack -j2 && \
cmake --install build-snowpack

In [ ]:
# @title Set up PATH to compiled code
!PATH="/home/app/usr/bin:${PATH}"
!LD_LIBRARY_PATH="/home/app/usr/lib:${LD_LIBRARY_PATH}"
!which snowpack

# The above cells have setup up SNOWPACK and made it ready to run. The below cells create the files, collect data, and run the snowpack model.

In [ ]:
#@title SNOWPACK Configuration Files Generation
"""
Generate .sno and .ini configuration files for SNOWPACK simulations
"""

#@markdown ## Station Configuration
station_id = "keystone_model"  #@param {"type":"string"}
station_name = "keystone alpine"  #@param {"type":"string"}
latitude = 39.56858687967004  #@param {"type":"number"}
longitude = -105.91900397453021  #@param {"type":"number"}
altitude = 3614  #@param {"type":"integer"}
timezone = -7  #@param {"type":"number"}
profile_date = "2024-11-01T00:00:00"  #@param {"type":"string"}

#@markdown ## Virtual Slopes Configuration
num_slopes = 5  #@param {"type":"number", "min":1, "max":10}
include_flat = True  #@param {"type":"boolean"}
default_slope_angle = 38.0  #@param {"type":"number", "min":0, "max":90}

#@markdown ## Slope Directions (degrees: 0=North, 90=East, 180=South, 270=West)
north_slope = True  #@param {"type":"boolean"}
east_slope = True  #@param {"type":"boolean"}
south_slope = True  #@param {"type":"boolean"}
west_slope = True  #@param {"type":"boolean"}
custom_directions = ""  #@param {"type":"string"} {description:"Comma-separated azimuth angles (e.g., 45,135,225,315)"}

#@markdown ## SNOWPACK Settings
meas_tss = "false"  #@param ["true","false"]
enforce_measured_snow_heights = "false"  #@param ["true","false"]

#@markdown ## Output Configuration
sno_directory = "/content/snowpack/Source/snowpack/tests/input"  #@param {"type":"string"}
ini_directory = "/content/snowpack/Source/snowpack/tests/keystone"  #@param {"type":"string"}
generate_config_files = True  #@param {"type":"boolean"}

# For local testing - use current directory
import os
if not os.path.exists("/content"):
    sno_directory = "."
    ini_directory = "."

def generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle):
    """Generate list of virtual slopes based on user selections"""
    slopes = []

    # Add flat slope if requested
    if include_flat:
        slopes.append((0.0, 0.0))

    # Add cardinal direction slopes
    if north_slope:
        slopes.append((default_slope_angle, 0.0))
    if east_slope:
        slopes.append((default_slope_angle, 90.0))
    if south_slope:
        slopes.append((default_slope_angle, 180.0))
    if west_slope:
        slopes.append((default_slope_angle, 270.0))

    # Add custom directions
    if custom_directions.strip():
        try:
            custom_angles = [float(x.strip()) for x in custom_directions.split(',')]
            for angle in custom_angles:
                if 0 <= angle <= 360:
                    slopes.append((default_slope_angle, angle))
        except ValueError:
            print("Warning: Invalid custom directions format. Use comma-separated numbers.")

    return slopes

def create_sno_content(station_id, station_name, longitude, latitude, altitude, timezone, profile_date, slope_angle, slope_azimuth):
    """Create .sno file content for a single slope"""

    content = f"""SMET 1.1 ASCII
[HEADER]
station_id       = {station_id}
station_name     = {station_name}
longitude        = {longitude}
latitude         = {latitude}
altitude         = {altitude}
nodata           = -999
tz               = {timezone}
source           = AWSOME
prototype        = SNOWPACK
ProfileDate      = {profile_date}
HS_Last          = 0.0000
SlopeAngle       = {slope_angle}
SlopeAzi         = {slope_azimuth}
nSoilLayerData   = 0
nSnowLayerData   = 0
SoilAlbedo       = 0.09
BareSoil_z0      = 0.020
CanopyHeight     = 0.00
CanopyLeafAreaIndex = 0.00
CanopyDirectThroughfall = 1.00
ErosionLevel     = 0
TimeCountDeltaHS = 0.000000
WindScalingFactor = 1.00

fields           = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo
[DATA]
"""

    return content

def create_ini_content(station_id, filename, snowfiles, meas_tss, enforce_measured_snow_heights):
    """Create .ini file content with multiple SNOWFILE entries"""

    content = f"""[General]
BUFFER_SIZE = 370
BUFF_BEFORE = 1.5

[Input]
COORDSYS = UTM
COORDPARAM = 13S
TIME_ZONE = -7

METEO = SMET
METEOPATH = ../input
METEOFILE1 = {filename}
"""

    # Add SNOWFILE entries for each slope
    for i, snowfile in enumerate(snowfiles, 1):
        content += f"SNOWFILE{i} = {snowfile}\n"

    content += f"""
[Output]
COORDSYS = UTM
COORDPARAM = 13S
TIME_ZONE = -7
METEOPATH = ./output
EXPERIMENT = res
SNOW_WRITE = false

TS_WRITE = true
TS_FORMAT = MET
TS_START = 0.0
TS_DAYS_BETWEEN = 0.125
PROF_WRITE = true
PROF_FORMAT = PRO
AGGREG_PRF = false
PROF_START = 0.0
PROF_DAYS_BETWEEN = 0.125

[Snowpack]
MEAS_TSS = {meas_tss.lower()}
ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights.lower()}
FORCING = ATMOS
SW_MODE = INCOMING
MEAS_INCOMING_LONGWAVE = false
HEIGHT_OF_WIND_VALUE = 4.5
HEIGHT_OF_METEO_VALUES = 4.5
ATMOSPHERIC_STABILITY = MO_MICHLMAYR
ROUGHNESS_LENGTH = 0.002
CALCULATION_STEP_LENGTH = 15.0
CHANGE_BC = false
THRESH_CHANGE_BC = -1.0
SNP_SOIL = false
SOIL_FLUX = false
GEO_HEAT = 0.06
CANOPY = false

[SnowpackAdvanced]
FIXED_POSITIONS = 0.25 0.5 1.0 -0.25 -0.10
SNOW_EROSION = true
WIND_SCALING_FACTOR = 1.0
NUMBER_SLOPES = {len(snowfiles)}
SNOW_REDISTRIBUTION = TRUE

[Filters]
ENABLE_METEO_FILTERS = true
PSUM::filter1 = min
PSUM::arg1::soft = true
PSUM::arg1::min = 0.0
TA::filter1 = min_max
TA::arg1::min = 240
TA::arg1::max = 320
RH::filter1 = min_max
RH::arg1::min = 0.01
RH::arg1::max = 1.2
RH::filter2 = min_max
RH::arg2::soft = true
RH::arg2::min = 0.05
RH::arg2::max = 1.0
ISWR::filter1 = min_max
ISWR::arg1::min = -10
ISWR::arg1::max = 1500
ISWR::filter2 = min_max
ISWR::arg2::soft = true
ISWR::arg2::min = 0
ISWR::arg2::max = 1500
RSWR::filter1 = min_max
RSWR::arg1::min = -10
RSWR::arg1::max = 1500
RSWR::filter2 = min_max
RSWR::arg2::soft = true
RSWR::arg2::min = 0
RSWR::arg2::max = 1500
ILWR::filter1 = min_max
ILWR::arg1::min = 188
ILWR::arg1::max = 600
ILWR::filter2 = min_max
ILWR::arg2::soft = true
ILWR::arg2::min = 200
ILWR::arg2::max = 400
TSS::filter1 = min_max
TSS::arg1::min = 200
TSS::arg1::max = 320
TSG::filter1 = min_max
TSG::arg1::min = 200
TSG::arg1::max = 320
HS::filter1 = min
HS::arg1::soft = true
HS::arg1::min = 0.0
HS::filter2 = rate
HS::arg2::max = 5.55e-5
VW::filter1 = min_max
VW::arg1::min = -2
VW::arg1::max = 70
VW::filter2 = min_max
VW::arg2::soft = true
VW::arg2::min = 0.0
VW::arg2::max = 50.0

[Interpolations1D]
MAX_GAP_SIZE = 86400
PSUM::resample1 = accumulate
PSUM::ARG1::period = 900
HS::resample1 = linear
HS::ARG1::MAX_GAP_SIZE = 43200
VW::resample1 = nearest
VW::ARG1::extrapolate = true
DW::resample1 = nearest
DW::ARG1::extrapolate = true

[Generators]
ILWR::generator1 = AllSky_LW
ILWR::arg1::type = Carmona
ILWR::arg1::shade_from_dem = FALSE
ILWR::arg1::use_rswr = FALSE
ILWR::generator2 = ClearSky_LW
ILWR::arg2::type = Dilley
"""

    return content

def get_slope_filename(angle, azimuth, slope_index, include_flat):
    """Get filename for slope naming: flat = 'keystone', others = 'keystone1', 'keystone2', etc."""
    if angle == 0.0:
        return "keystone"  # Flat slope gets no number
    else:
        # For numbered slopes, determine the number based on position
        if include_flat:
            # If flat is included, non-flat slopes start from index 1
            return f"keystone{slope_index}"
        else:
            # If no flat, start numbering from 1
            return f"keystone{slope_index + 1}"

def main():
    """Main execution function"""

    print("SNOWPACK Configuration Files Generation")
    print("=" * 50)
    print(f"Station: {station_name} ({station_id})")
    print(f"Location: {latitude:.6f}°N, {longitude:.6f}°W, {altitude}m")
    print(f"Sno Directory: {sno_directory}")
    print(f"Ini Directory: {ini_directory}")
    print("=" * 50)

    if not generate_config_files:
        print("Configuration generation disabled. Set 'generate_config_files' to True to generate files.")
        return

    # Create output directories
    os.makedirs(sno_directory, exist_ok=True)
    os.makedirs(ini_directory, exist_ok=True)

    # Generate slopes
    slopes = generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle)

    # Limit to requested number of slopes
    if len(slopes) > num_slopes:
        slopes = slopes[:num_slopes]
        print(f"Warning: Limited to {num_slopes} slopes as requested")

    # Create multiple .sno files and collect snowfile names
    snowfiles = []
    sno_files_created = []

    for i, (angle, azimuth) in enumerate(slopes):
        # Get filename based on new naming convention
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat) + ".sno"
        snowfiles.append(sno_filename)

        # Create .sno content for this slope
        sno_content = create_sno_content(station_id, station_name, longitude, latitude, altitude, timezone, profile_date, angle, azimuth)

        # Write .sno file to input directory
        sno_filepath = os.path.join(sno_directory, sno_filename)
        with open(sno_filepath, "w") as f:
            f.write(sno_content)

        sno_files_created.append(sno_filepath)

    # Create .ini content with multiple SNOWFILE entries
    ini_content = create_ini_content(station_id, f"{station_id}.smet", snowfiles, meas_tss, enforce_measured_snow_heights)

    # Write .ini file to keystone directory
    ini_filename = os.path.join(ini_directory, f"{station_id}.ini")
    with open(ini_filename, "w") as f:
        f.write(ini_content)

    # Display results
    print(f"\n Configuration files created:")
    print(f"   .ini file: {ini_filename}")
    print(f"   .sno files:")
    for sno_file in sno_files_created:
        print(f"     {sno_file}")

    print(f"\nVirtual slopes configured: {len(slopes)}")
    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat)
        if angle == 0.0:
            print(f"   Slope {i+1}: Flat (0°) -> {sno_filename}.sno")
        else:
            direction = ""
            if azimuth == 0.0:
                direction = "North"
            elif azimuth == 90.0:
                direction = "East"
            elif azimuth == 180.0:
                direction = "South"
            elif azimuth == 270.0:
                direction = "West"
            else:
                direction = f"{azimuth}°"
            print(f"   Slope {i+1}: {angle}° slope facing {direction} -> {sno_filename}.sno")

    print(f"\nSNOWPACK settings:")
    print(f"   MEAS_TSS = {meas_tss}")
    print(f"   ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights}")
    print(f"   NUMBER_SLOPES = {len(slopes)}")

    print("\n🎉 SNOWPACK configuration generation complete!")

# Run the main function
if __name__ == "__main__":
    main()

In [ ]:
# @title SMET Handling and API Setup
def create_smet_from_weather_data(weather_df, output_path, station_id, station_name, latitude, longitude, altitude):
    """
    Create SMET file from weather DataFrame
    """
    # Ensure timestamp is datetime
    if 'timestamp' in weather_df.columns:
        weather_df = weather_df.copy()
        weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
    else:
        weather_df = weather_df.copy()
        weather_df.reset_index(inplace=True)
        weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])

    # Define fields in order
    fields = ['timestamp', 'TA', 'RH', 'TSG', 'VW', 'DW', 'ISWR', 'PSUM']
    if 'HS' in weather_df.columns:
        fields.append('HS')

    # Create output directory if needed
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(output_path, 'w') as f:
        # Write SMET header
        f.write("SMET 1.1 ASCII\n")
        f.write("[HEADER]\n")
        f.write(f"station_id = {station_id}\n")
        f.write(f"station_name = {station_name}\n")
        f.write(f"latitude = {latitude:.10f}\n")
        f.write(f"longitude = {longitude:.10f}\n")
        f.write(f"altitude = {altitude}\n")
        f.write("nodata = -777\n")
        f.write("Tz = -7\n")
        f.write(f"fields = {'\t'.join(fields)}\n")

        # Units
        field_count = len(fields)
        units_offset = ["0"] * field_count
        units_multiplier = ["1"] * field_count
        f.write(f"units_offset     = {' '.join(units_offset)}\n")
        f.write(f"units_multiplier = {' '.join(units_multiplier)}\n")

        # Data section
        f.write("[DATA]\n")

        # Write data rows
        for _, row in weather_df.iterrows():
            values = []
            for field in fields:
                if field == 'timestamp':
                    values.append(row[field].strftime('%Y-%m-%dT%H:%M:%S'))
                else:
                    val = row.get(field, -777)
                    if pd.isna(val):
                        val = -777
                    values.append(f"{val:.2f}")
            f.write("\t".join(values) + "\n")

def fetch_openmeteo_historical(lat, lon, start_date, end_date, model="gfs_hrrr", elevation=None):
    """
    Fetch historical forecast data from Open-Meteo API
    """
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "wind_direction_10m",
            "snow_depth",
            "direct_radiation",
            "precipitation"
        ],
        "models": model,
        "wind_speed_unit": "ms",
    }

    if elevation is not None:
        params["elevation"] = elevation

    print(f"Fetching {model} data for {lat}, {lon} from {start_date} to {end_date}")

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation: {response.Elevation()} m asl")

        # Process hourly data
        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()

        # Create time index
        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        # Create DataFrame
        df = pd.DataFrame({
            'timestamp': time_index,
            'TA': hourly_temperature_2m + 273.15,  # Convert to Kelvin
            'RH': hourly_relative_humidity_2m / 100.0,  # Convert to fraction
            'VW': hourly_wind_speed_10m,  # Already in m/s
            'DW': hourly_wind_direction_10m,  # Already in degrees
            'HS': hourly_snow_depth,  # Snow depth in meters
            'ISWR': hourly_direct_radiation,  # Direct radiation in W/m²
            'PSUM': hourly_precipitation,  # Precipitation in mm
        })

        # Add missing fields
        df['TSG'] = 273.15  # Ground temperature (assumed)

        # Handle missing values
        df = df.replace([np.inf, -np.inf], np.nan)

        print(f"Fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


In [ ]:
# @title SMET Generation Parameters
# @markdown Configure your parameters below and run this cell to generate SMET files

# @markdown Location Settings
latitude = 39.56858687967004 # @param {type:"number"}
longitude = -105.91900397453021 # @param {type:"number"}
altitude = 3614 # @param {type:"integer"}

# @markdown Station Information
station_name = "keystone_model" # @param {type:"string"}

# @markdown Time Period
start_date = "2024-11-01" # @param {type:"date"}
end_date = "2025-04-30" # @param {type:"date"}

# @markdown Weather Models
model_selection = "ifs" # @param ["nbm","ifs"]

# @markdown Generate SMET Files
generate_files = True # @param {type:"boolean"}

print("SMET Generation Parameters")
print("=" * 50)
print(f"Location: {latitude}, {longitude} (altitude: {altitude}m)")
print(f"Station: {station_name}")
print(f"Period: {start_date} to {end_date}")
print(f"Model: {model_selection}")
print(f"Generate: {generate_files}")
print("=" * 50)


In [ ]:
# @title Generate SMET Files
# @markdown Run this cell to generate SMET files with the parameters above

# Hardcoded output path
output_directory = "/content/snowpack/Source/snowpack/tests/input"

# Model mapping
model_mapping = {
    "nbm": "ncep_nbm_conus",
    "ifs": "ecmwf_ifs"
}

if generate_files:
    print("Starting SMET generation...")

    # Parse model selection
    selected_models = [model_selection]

    # Validate and map models
    valid_models = ['nbm', 'ifs']
    selected_models = [model for model in selected_models if model in valid_models]

    if not selected_models:
        print("No valid models selected!")
        print(f"Available models: {', '.join(valid_models)}")
    else:
        print(f"Processing {len(selected_models)} model(s): {', '.join(selected_models)}")
        print("="*50)

        successful_files = []

        for model in selected_models:
            print(f"\nProcessing {model}...")

            # Map to Open-Meteo model name
            openmeteo_model = model_mapping[model]

            # Fetch data
            df = fetch_openmeteo_historical(
                lat=latitude,
                lon=longitude,
                start_date=start_date,
                end_date=end_date,
                model=openmeteo_model,
                elevation=altitude
            )

            if df is None or df.empty:
                print(f"Failed to fetch data for {model}")
                continue

            # Create output filename with simplified naming
            output_file = f"{output_directory}/{station_name}.smet"
            # could add in _{model} if wanted to specify

            try:
                # Generate SMET file
                create_smet_from_weather_data(
                    weather_df=df,
                    output_path=output_file,
                    station_id=f"{station_name}_{model}",
                    station_name=f"{station_name}_{model}",
                    latitude=latitude,
                    longitude=longitude,
                    altitude=altitude
                )

                print(f"SMET file created: {output_file}")
                print(f"Records: {len(df)}")
                print(f"Period: {df['timestamp'].min()} to {df['timestamp'].max()}")

                successful_files.append(output_file)

            except Exception as e:
                print(f"Error creating SMET for {model}: {e}")
                continue

        print(f"\nSuccessfully generated {len(successful_files)} SMET files:")
        for file in successful_files:
            print(f"  {file}")

        if successful_files:
            print(f"\nFiles saved to: {output_directory}")
else:
    print("Generation disabled. Set 'generate_files' to True to generate SMET files.")

In [ ]:
# @title Run this cell to run SNOWPACK
!cd /content/snowpack/Source/snowpack/tests/keystone/ && \
/usr/local/bin/snowpack -c /content/snowpack/Source/snowpack/tests/keystone/keystone_model.ini -e 2025-04-01T00:00


In [ ]:
# @title Download .pro files and Open niViz
# @markdown # 5 files will be download.
# @markdown # 1 = flat, 2=N, 3=E, 4=S, 5=W
import glob, shutil, os
from IPython.display import FileLink, Markdown
from google.colab import files

# Locate all .pro files
pro_files = glob.glob('/content/snowpack/Source/snowpack/tests/keystone/output/*.pro')

if pro_files:
    # Copy all files to a simple location
    downloaded_files = []
    for pro_file in pro_files:
        filename = pro_file.split('/')[-1]
        shutil.copy(pro_file, f'/content/{filename}')
        downloaded_files.append(filename)

    # Create a zip file with all .pro files
    import zipfile
    zip_filename = 'snowpack_profiles.zip'
    with zipfile.ZipFile(f'/content/{zip_filename}', 'w') as zipf:
        for filename in downloaded_files:
            zipf.write(f'/content/{filename}', filename)

    # Download the zip file
    files.download(f'/content/{zip_filename}')

    # Show how to open in niViz
    display(Markdown(f"""
    ---
    {len(downloaded_files)} SNOWPACK profile files have been downloaded as **`{zip_filename}`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    """))

    for filename in downloaded_files:
        display(Markdown(f"   - **`{filename}`**"))

else:
    display(Markdown("No .pro files found in the output directory."))